In [1]:
import sys

# Add the project root directory to Python path
sys.path.append('/Users/alexanderithakis/Documents/GitHub/D4PG-QR-FRM')

# Relaod the src modules to ensure the latest changes are used
import importlib
import src.fdm_swing_pricer
import src.swing_contract
importlib.reload(src.fdm_swing_pricer)
importlib.reload(src.swing_contract)
from src.fdm_swing_pricer import price_swing_option_fdm
from src.swing_contract import SwingContract
from src.simulate_hhk_spot import no_seasonal_function

# Monthly Swing Option Contract Parameters
swing_contract = SwingContract(
    q_min=0.0,                  # No minimum exercise requirement per day
    q_max=2.0,                  # Max 2 units per day (reasonable daily limit)
    Q_min=0.0,                  # No minimum total exercise requirement
    Q_max=20.0,                 # Max 20 units total over the month (10 days worth)
    strike=1.0,                 # At-the-money strike
    maturity=1/12,              # 1 month = 1/12 year
    n_rights=22,                # ~22 trading days in a month
    r=0.05,                     # 5% annual risk-free rate
    min_refraction_days=0       # No refraction period (can exercise daily)
)

# Market Process Parameters (HHK model - monthly calibration)
stochastic_process_params = {
    'S0': 1.0,                  # Initial spot price
    'T': swing_contract.maturity, # 1/12 year
    'n_steps': swing_contract.n_rights - 1,  # 21 steps (22 decision points)
    'n_paths': 8192,            # Evaluation paths for LSM
    'alpha': 12.0,              # Higher mean reversion for monthly timeframe
    'sigma': 1.2,               # Moderate volatility for monthly period
    'beta': 150.0,              # Jump decay rate
    'lam': 6.0,                 # Jump intensity (6 jumps per year average)
    'mu_J': 0.3,                # Mean jump size (30%)
    'f': no_seasonal_function   # No seasonal function
}

price_swing_option_fdm(
    contract=swing_contract,
    stochastic_process_params=stochastic_process_params,
    tGrid=25,
    xGrid=25,
    yGrid=50,  # Adjusted yGrid for better performance
)

1.2129091862251442